In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Cross at no threshold
cross_orig = pd.read_csv('cross.tsv', sep='\t', header=None)
cross_orig.columns = ['first', 'second', 'label']
cross_orig

,first,second,label
0,Q95QA6,P90735,1
1,Q95QA6,Q17573,1
2,Q95QA6,Q11103,1
3,Q95QA6,O16510,1
4,Q95QA6,Q17695,1
...,...,...,...
65913,P18471,P02932,0
65914,P18472,P02930,0
65915,P18035,P41066,0
65916,P10904,P41069,0


In [2]:
# Create dictionary of uni id and seq of E.coli, D.melanogaster, E.coli
# E.coli
df_uni_seq_ecoli = pd.read_csv('dict_seq_E.coli.tsv', sep='\t', header=None)
df_uni_seq_ecoli.columns = ['uni', 'seq']
dict_uni_seq_ecoli = {id: seq for id, seq in zip(list(df_uni_seq_ecoli['uni'].values), list(df_uni_seq_ecoli['seq'].values))}

# D.melanogaster
df_uni_seq_dmelanogaster = pd.read_csv('dict_seq_D.melanogaster.tsv', sep='\t', header=None)
df_uni_seq_dmelanogaster.columns = ['uni', 'seq']
dict_uni_seq_dmelanogaster = {id: seq for id, seq in zip(list(df_uni_seq_dmelanogaster['uni'].values), list(df_uni_seq_dmelanogaster['seq'].values))}

# C.elegans
df_uni_seq_celegans = pd.read_csv('dict_seq_C.elegans.tsv', sep='\t', header=None)
df_uni_seq_celegans.columns = ['uni', 'seq']
dict_uni_seq_celegans = {id: seq for id, seq in zip(list(df_uni_seq_celegans['uni'].values), list(df_uni_seq_celegans['seq'].values))}


In [4]:
def create_species_dataset(set_uni_seq_species):
    index = []
    for i in range(len(cross_orig)):
        if cross_orig['first'][i] in set_uni_seq_species and cross_orig['second'][i] in set_uni_seq_species:
            index.append(i)
    return index

# E.coli
ecoli_index = create_species_dataset(set_uni_seq_ecoli)
ecoli_dataset = cross_orig.iloc[ecoli_index, :].reset_index(drop=True)
print("Ecoli: \n",ecoli_dataset)

# D.melanogaster
dmelanogaster_index = create_species_dataset(set_uni_seq_dmelanogaster)
dmelanogaster_dataset = cross_orig.iloc[dmelanogaster_index, :].reset_index(drop=True)
print("D.melanogaster: \n")
dmelanogaster_dataset

# C.elegans
celegans_index = create_species_dataset(set_uni_seq_celegans)
celegans_dataset = cross_orig.iloc[celegans_index, :].reset_index(drop=True)
print("C.elegans: \n")
celegans_dataset    

NameError: name 'set_uni_seq_ecoli' is not defined

In [5]:
ecoli_dataset.to_csv('pairs_ecoli.tsv', header=None, index=None, sep='\t')
dmelanogaster_dataset.to_csv('pairs_dmelanogaster.tsv', header=None, index=None, sep='\t')
celegans_dataset.to_csv('pairs_celegans.tsv', header=None, index=None, sep='\t')

In [5]:
# Set of all proteins organisms for clustering with different thresholds
set_protein_all_organisms = frozenset(list(ecoli_dataset['first'])+list(ecoli_dataset['second'])+list(celegans_dataset['first'])+list(celegans_dataset['second'])+
list(dmelanogaster_dataset['first'])+list(dmelanogaster_dataset['second']))

# Write to file all organisms proteins
f = open('proteins_all_organisms.txt', 'w')
for protein in set_protein_all_organisms:
    f.write(protein+'\n')


# Concatenate 3 organisms dataframe
cross_dataset = pd.concat([ecoli_dataset, dmelanogaster_dataset, celegans_dataset])
cross_dataset

NameError: name 'ecoli_dataset' is not defined

In [24]:
dict_uni_seq = dict(dict_uni_seq_celegans)
dict_uni_seq.update(dict_uni_seq_dmelanogaster)
dict_uni_seq.update(dict_uni_seq_ecoli)
dict_uni_seq

{'Q9U3C8': 'MNRLKSDQKTKLRQFVQWTQVTEAVSLNFLAKANWNIEYAMTLYFDNPNLFAGSTPQPSVDRSNIERLFNQYVDPKDKVGEKRMGPHGINRLLTDLGYEATDRRVLVLAWKFTAQTQCEFSLDEWVKGMTALQADTVQNLRQRIDSINSGLESDKAKFHELYLFAFNYAKSAACRNLDLETAICCWDVLFGQRSTIMTQWIDFLWAQENAAASRLAQNVGASNAKQFKSVWISRDTWNLFWDFILLSKPDLSDYDDEGAWPVLIDQFVDYCRENLNYPKPGNASNDQQMETPSYY',
 'P34441': 'MDFSIVKTQILNNRRTFRTPFKVSSMCFSSQNDLIALGSKTGEILLKRTSWKMIWKTNINMIQAVGTECKLDSSVSALHFSPDGRFLAAATSKGIIHLLDVETGKVRFSVKAASEKIAKLHWNCVREKPFISNLGEFTTRIKNVEAIEGAIELAETTPNISQEEIAFVYQRLDEDGSSFKHEDAHKESLERTLISTETFRESLQNTILLATDDMDSKIIVLVAGVFPYMEIDISDTLLQYNQSLLMLYDMHYSSAFGGVSFLATTYGPFLDCKQNELKPPGAEPKKDGQGCHTLLFNVKLNINSSLWDTALRYIRLLFGFNLYSISLETTKRNWEEQIDNLHSLFDTKTKAVKIGNVLLEMLLSGSTDAAGEAFLERGLGTDGLDKIELFATKHMPEVCRIARGQLSTSARNLCFQRCEFSTSLSRYAKFIQLKDDDSFLYDEDPPAYLSESNIWLNTLEEKINILDMKTRHLGIQCLTMMQELGHLVKWISMTKPFAKTMKVNALMKIKRMNIAKILLYIVRNFIPDPEAVKDIENRLFNLKELVSKQKKKDLEDKFDEVEYLYRLERIAEVERKKLIETFHEKFRFADLDDDDLTPLLFQELDHPIDIFTDSMMEQNDSSPFLEEDEGEPEQEEQKPDEEPEPEGEPQPACDLDRVGSFFEKELSQKALEVLPKCD

In [28]:
# Write dictionary to fasta file
ofile = open("all_organism_proteins.fasta", "w")

for key, value in dict_uni_seq.items():

    ofile.write(">" + key + "\n" +value + "\n")

#do not forget to close it

ofile.close()

In [6]:
### interaction pairs in pairs_ecoli.tsv, pairs_dmelanogaster.tsv, pairs_celegans.tsv, dictionary in uni_seq_species.txt